In [22]:
from profilescraper import query_profile
import pandas as pd
import itertools

import os

from glob import glob

from proxy_utils import proxy_dict, get_proxies

In [13]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
def profiles_to_pandas(profiles):
    userDf = pd.DataFrame(columns=['username', 'location', 'has_location', 'age', 'is_verified', 'total_tweets', 'total_following', 'total_followers', 'total_likes', 'total_moments', 'total_lists', 'has_avatar', 'has_background', 'is_protected', 'profile_modified', 'tweets'])
    tweetDf = pd.DataFrame(columns=['User', 'ID', 'Tweet', 'Time', 'Likes', 'Replies', 'Retweet'])

    for profile in profiles:   
        for tweet in profile.tweets:
            tweetDf = tweetDf.append({'User': profile.username, 'ID': tweet.id, 'Tweet': tweet.text, 'Time': tweet.timestamp, 'Likes': tweet.likes, 'Replies': tweet.replies, 'Retweet': tweet.retweets}, ignore_index=True)

        userDf = userDf.append({'username':profile.username, 'location':profile.location, 'has_location':profile.has_location, 'age':profile.age, 'is_verified':profile.is_verified, 'total_tweets':profile.total_tweets, 'total_following':profile.total_following, 'total_followers':profile.total_followers, 'total_likes':profile.total_likes, 'total_moments':profile.total_moments, 'total_lists':profile.total_lists, 'has_avatar':profile.has_avatar, 'has_background':profile.has_background, 'is_protected':profile.is_protected, 'profile_modified':profile.profile_modified}, ignore_index=True)

    tweetDf = tweetDf.to_csv('profiledata/userTweets.csv', index=None)
    userDf['username'].to_csv('profiledata/extractedUsers.csv', index=None, mode='a')
    userDf.to_csv('profiledata/userData.csv', index=None, mode='a')
    
    print("Saved to userTweets.csv and extractedUsers.csv")

In [46]:
def scrape_list(currList, poolsize, proxy, count):
        
    if (len(currList) > 0):
        profiles = query_profile(currList, poolsize=poolsize, proxy=proxy)
        profiles_to_pandas(profiles)

        count += 1
    
    return count

In [47]:
def perform_extraction(coinname, poolsize=20):
    proxies = get_proxies()
    proxySize = len(proxies)
    
    users = list(set(pd.read_csv('{}/extracted/combined.csv'.format(coinname), dtype=str)['User']))
    
    try:
        alreadyRead = pd.read_csv('profiledata/extractedUsers.csv', header=None)[0]
    except FileNotFoundError:
        logging.info("Already extracted users not found - Starting from a clean slate")
        os.mknod("profiledata/extractedUsers.csv")
        alreadyRead = pd.Series()
        
    
    uniqueUsers = list(set(users) - set(alreadyRead))
    
    print("File contains {} data. Scraping for {} after cache".format(len(users), len(uniqueUsers)))
    
    oldi = 0
    count = 0
    
    for i in range(0, len(uniqueUsers), poolsize*5):
        count = scrape_list(uniqueUsers[oldi:i], poolsize=poolsize, proxy=proxies[count], count=count)
        
        if (count >= proxySize):
            count = 0
        
        logging.info("Done {} of {}".format(i, len(uniqueUsers)))
        oldi = i
    
    scrape_list(uniqueUsers[i:], poolsize=poolsize, proxy=None, count=count)

In [ ]:
for files in glob(os.getcwd() + "/*"):
    if (os.path.exists(files + "/extracted")):
        perform_extraction(files.split("/")[-1], poolsize=20)

INFO:root:Already extracted users not found - Starting from a clean slate
INFO:root:Done 0 of 44380


File contains 44380 data. Scraping for 44380 after cache


INFO:root:Querying https://twitter.com/PEIBistro
INFO:root:Querying https://twitter.com/BritOakPub_SE3
INFO:root:Querying https://twitter.com/RoxieBrookshire
INFO:root:Querying https://twitter.com/8CountAudio
INFO:root:Querying https://twitter.com/BitcoinBarons
INFO:root:Querying https://twitter.com/InvestFractal
INFO:root:Querying https://twitter.com/booksbyRoger
INFO:root:Querying https://twitter.com/INFORUMsf
INFO:root:Querying https://twitter.com/bitcoinpoet
INFO:root:Querying https://twitter.com/OneandOnlyGregV
INFO:root:Querying https://twitter.com/mrslynch5th
INFO:root:Querying https://twitter.com/Woggieeee
INFO:root:Querying https://twitter.com/ProjManagSydney
INFO:root:Querying https://twitter.com/coexistkittens
INFO:root:Querying https://twitter.com/RosCeilteach
INFO:root:Querying https://twitter.com/Rhogew
INFO:root:Querying https://twitter.com/JanPotenciano
INFO:root:Querying https://twitter.com/dhume
INFO:root:Querying https://twitter.com/hthhglobal
INFO:root:Querying http

INFO:root:Got 87 profiles (1 new).
INFO:root:Got 88 profiles (1 new).
INFO:root:Got 89 profiles (1 new).
INFO:root:Got 90 profiles (1 new).
INFO:root:Got 91 profiles (1 new).
INFO:root:Got 92 profiles (1 new).
INFO:root:Got 93 profiles (1 new).
INFO:root:Got 94 profiles (1 new).
INFO:root:Got 95 profiles (1 new).
INFO:root:Got 95 profiles (0 new).
INFO:root:Got 96 profiles (1 new).
INFO:root:Done 100 of 44380


Saved to userTweets.csv and extractedUsers.csv


...

Saved to userTweets.csv and extractedUsers.csv


INFO:root:Querying https://twitter.com/cryproking225
INFO:root:Querying https://twitter.com/QDPurdu
INFO:root:Querying https://twitter.com/darkbeautymag
INFO:root:Querying https://twitter.com/waiwaipang
INFO:root:Querying https://twitter.com/abdulnaem
INFO:root:Querying https://twitter.com/cleveleysnews
INFO:root:Querying https://twitter.com/BrianSurreal
INFO:root:Querying https://twitter.com/syscoin
INFO:root:Querying https://twitter.com/dj_mosfett
INFO:root:Querying https://twitter.com/marianaatencio
INFO:root:Querying https://twitter.com/proactiff
INFO:root:Querying https://twitter.com/CKTSUNE
INFO:root:Querying https://twitter.com/CharweeMonster
INFO:root:Querying https://twitter.com/VicCarpetCare
INFO:root:Querying https://twitter.com/TheDustyBC
INFO:root:Querying https://twitter.com/NeushanPradhan
INFO:root:Querying https://twitter.com/CAA_Official
INFO:root:Querying https://twitter.com/janelouise0911
INFO:root:Querying https://twitter.com/JacobCMedia
INFO:root:Querying https://t

INFO:root:Got 84 profiles (1 new).
INFO:root:Got 85 profiles (1 new).
INFO:root:Got 86 profiles (1 new).
INFO:root:Got 87 profiles (1 new).
INFO:root:Got 88 profiles (1 new).
INFO:root:Got 89 profiles (1 new).
INFO:root:Got 90 profiles (1 new).
INFO:root:Got 91 profiles (1 new).
INFO:root:Got 92 profiles (1 new).
INFO:root:Got 93 profiles (1 new).
INFO:root:Got 94 profiles (1 new).
INFO:root:Done 800 of 44380


Saved to userTweets.csv and extractedUsers.csv


INFO:root:Querying https://twitter.com/ExplorerMD
INFO:root:Querying https://twitter.com/PleadingLake
INFO:root:Querying https://twitter.com/RiseAbovePumps
INFO:root:Querying https://twitter.com/doc_bitcoin
INFO:root:Querying https://twitter.com/khris_gallo
INFO:root:Querying https://twitter.com/BrianKarem
INFO:root:Querying https://twitter.com/demipip
INFO:root:Querying https://twitter.com/DrVikIam
INFO:root:Querying https://twitter.com/BrotherJesse
INFO:root:Querying https://twitter.com/604_stats
INFO:root:Querying https://twitter.com/quantumhealer1
INFO:root:Querying https://twitter.com/PSmith3_
INFO:root:Querying https://twitter.com/DandalBs
INFO:root:Querying https://twitter.com/Tatum_Flynn
INFO:root:Querying https://twitter.com/NickcCollins
INFO:root:Querying https://twitter.com/ChampField_SU
INFO:root:Querying https://twitter.com/JeremiahBrent
INFO:root:Querying https://twitter.com/NeedMoahCowbell
INFO:root:Querying https://twitter.com/v_rrose
INFO:root:Querying https://twitter.